In [ ]:
# Needed for dependencies
!sudo apt install python3.11-dev && sudo apt-get install build-essential -y

In [1]:
!python3 -m venv .venv

In [2]:
!pip install jupyter

In [3]:
!pip install ipywidgets widgetsnbextension pandas-profiling

In [10]:
!pip install accelerate==0.31.0

  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.32.1
    Uninstalling accelerate-0.32.1:
      Successfully uninstalled accelerate-0.32.1


In [4]:
!pip install transformers==4.35.0 chromadb==0.4.6 accelerate tqdm 

# Creación de un Entorno de Python

En este documento, vamos a explicar cómo configurar un nuevo entorno de Python utilizando tanto `venv` como `conda`.

## Usando venv

`venv` es una herramienta que viene preinstalada con Python (versión 3.3 o superior) y permite crear entornos virtuales de Python aislados.

### Pasos

1. **Crear un entorno virtual:** Abre la terminal y ejecuta el siguiente comando en el directorio donde desees crear el entorno virtual.

   ```bash
   python3 -m venv nombre_del_entorno


2. **Activar el entorno virtual:** Utiliza el siguiente comando para activar el entorno.


  Linux/Mac:

```bash
    source nombre_del_entorno/bin/activate
```


   Windows:

```bash
    .\nombre_del_entorno\Scripts\activate
```


3. **Instalar dependencias desde requirements.txt con pip:** Una vez activado el entorno, puedes instalar todas las dependencias necesarias desde un archivo `requirements.txt` utilizando:

   ```bash
   pip install -r requirements.txt

4. **Desactivar el entorno virtual:** Para salir del entorno virtual, simplemente ejecuta:

```bash
   deactivate

## Usando Conda
conda es un sistema de gestión de paquetes y de entorno que puede instalar paquetes de diferentes lenguajes.

### Pasos
1. **Instalar Conda:** Puedes descargar e instalar Conda desde [este enlace](https://www.anaconda.com/download).

2. **Crear un entorno Conda:** Para crear un nuevo entorno con Conda, abre la terminal y ejecuta:

```bash
    conda create --name nombre_del_entorno python=3.9 ipykernel
```

Para esta biblioteca, vamos a usar python 3.9 y para interactuar con Jupyter Notebooks vamos a usar "ipykernel" o "jupyter"

3. **Activar el entorno Conda:** Utiliza el siguiente comando para activar el entorno.

```bash
    conda activate nombre_del_entorno
```
4. **Instalar dependencias desde requirements.txt con pip:** Al igual que con venv, puedes instalar las dependencias necesarias desde un archivo requirements.txt en el entorno Conda activado utilizando:
```bash
    pip install -r requirements.txt
```

5. Desactivar el entorno Conda: Para salir del entorno Conda, simplemente ejecuta:

```bash
    conda deactivate
```

In [78]:
# 1️⃣ Importar librerías
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import os
import numpy as np
import uuid

# Modelo de embeddings
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(model_name)

def get_embeddings(text):
    return model.encode(text).tolist()

# Inicializar ChromaDB con persistencia
client = chromadb.PersistentClient(path="./chroma_store")
collection = client.get_or_create_collection(name="documentos_ia")

def cargar_documentos():
    docs_path = "docs/"
    documentos = []
    metadatos = []
    ids = []

    for filename in os.listdir(docs_path):
        if filename.endswith(".txt"):
            with open(os.path.join(docs_path, filename), 'r', encoding="utf-8") as f:
                text = f.read().strip()
                chunks = [text[i:i+500] for i in range(0, len(text), 500)]
                
                for i, chunk in enumerate(chunks):
                    documentos.append(chunk)
                    metadatos.append({"source": filename})
                    ids.append(str(uuid.uuid4()))  # ID único
    
    return documentos, metadatos, ids

def crear_base_datos():
    docs, metas, ids = cargar_documentos()
    embeddings = [get_embeddings(d) for d in docs]
    collection.add(documents=docs, embeddings=embeddings, metadatas=metas, ids=ids)
    print("✅ Base de datos creada con éxito.")

# --- CRUD FUNCIONES ---

def create_example(new_doc):
    """Añadir un documento nuevo"""
    
    # Verificar si el documento está vacío
    if not new_doc.strip():  # Si el texto está vacío o solo tiene espacios en blanco
        print("❌ El documento está vacío. No se puede agregar.")
        return

    embedding = get_embeddings(new_doc)
    #embedding = [] # Array vacio para pruebas
    doc_id = str(uuid.uuid4())
    metadata = {"source": "nuevo_documento.txt"}
    
    print(f"Total original de documentos en la colección: {collection.count()}")

    collection.add(
        documents=[new_doc],
        embeddings=[embedding],
        metadatas=[metadata],
        ids=[doc_id]
    )
    print(f"✅ Documento añadido con ID: {doc_id}")
    
    print(f"Total de documentos en la colección actualizado: {collection.count()}")


def read_example(query):
    """Consultar documentos similares"""
    query_emb = get_embeddings(query)
    results = collection.query(
        query_embeddings=[query_emb],
        n_results=1
    )
    
    if results["documents"]:
        print(f"📄 Mejor coincidencia: {results['documents'][0][0]}")
        print(f"📁 Fuente: {results['metadatas'][0][0]['source']}")
        print(f"🆔 ID: {results['ids'][0][0]}")
    else:
        print("⚠️ No se encontraron resultados.")

    # Verificar la cantidad de resultados obtenidos por la busqueda
    if len(results["ids"][0]) == 1:
        print("✅ Solo se encontró un documento similar.")
    else:
        print(f"❌ Se encontraron {len(results['ids'][0])} documentos similares.")
    
    total_documents = collection.count()
    print(f"Total de documentos en la colección: {total_documents}")


def get_documents_by_id(doc_id):
    """Obtener un documento por ID"""
    results = collection.get(ids=[doc_id], include=["documents", "metadatas", "embeddings"])
    
    if results["documents"]:
        print(f"📄 Documento: {results['documents'][0]}")
        print(f"📁 Fuente: {results['metadatas'][0]['source']}")
    else:
        print("⚠️ Documento no encontrado.")

def update_example(old_id, new_text):
    """Actualizar un documento (eliminar y reinsertar)"""
    collection.delete(ids=[old_id])
    
    new_id = str(uuid.uuid4())
    embedding = get_embeddings(new_text)
    metadata = {"source": "documento_actualizado.txt"}
    
    collection.add(
        documents=[new_text],
        embeddings=[embedding],
        metadatas=[metadata],
        ids=[new_id]
    )
    print(f"♻️ Documento actualizado. Nuevo ID: {new_id}")

def delete_example(doc_id):
    if doc_id == 'all_documents':
        neutral_embedding = [0.0] * 384
        results = collection.query(
            query_embeddings=[neutral_embedding],  # embedding neutro para traer resultados.
            n_results=1000 
        )
        doc_ids = results["ids"][0] 
        if doc_ids:
            """Eliminar todos los documentos"""
            print(f"Total de documentos en la colección antes del borrado: {collection.count()}")
            collection.delete(ids=doc_ids)
            print(f"🗑️ Todos los documentos fueran eliminados. Total actualizado: {collection.count()}")
        else:
            print("🗑️ No hay documentos a eliminar.")
    else:
        """Eliminar por ID"""
        print(f"Total de documentos en la colección antes del borrado: {collection.count()}")
        collection.delete(ids=[doc_id])
        print(f"🗑️ Documento con ID '{doc_id}' eliminado.")
        print(f"Total de documentos en la colección despues de borrar: {collection.count()}")
    

print('listo')

listo


In [74]:
crear_base_datos()

✅ Base de datos creada con éxito.


In [75]:
get_embeddings("Reynaldo González es un ingeniero de software con 10 años de experiencia en desarrollo web y móvil.")

[-0.09879040718078613,
 0.011900072917342186,
 -0.07934798300266266,
 -0.07173144817352295,
 0.048498984426259995,
 -0.08828755468130112,
 0.04442286118865013,
 0.06878753751516342,
 -0.04471665248274803,
 -0.01907443068921566,
 0.009390952065587044,
 0.055638331919908524,
 0.02038574405014515,
 0.005857574287801981,
 0.0571175143122673,
 0.034097906202077866,
 -0.03325342759490013,
 0.008881207555532455,
 0.022362619638442993,
 -0.06751403212547302,
 0.11720023304224014,
 -0.02992631494998932,
 0.0018127455841749907,
 -0.007679759059101343,
 -0.014972859062254429,
 0.0011125863529741764,
 0.0069056181237101555,
 -0.01639213040471077,
 -0.025348804891109467,
 -0.09709930419921875,
 0.03483699634671211,
 0.06626581400632858,
 0.07979559153318405,
 0.05058017000555992,
 -0.024885039776563644,
 -0.004187216982245445,
 0.021157730370759964,
 -0.058077335357666016,
 -0.07937502861022949,
 0.012050786055624485,
 -0.1423080414533615,
 0.029957273975014687,
 -0.003748437622562051,
 -0.07423300

In [50]:
results = collection.get(include=["embeddings", "documents", "metadatas"])

In [76]:
for id, content, metadata, embedding in zip(results["ids"], results["documents"], results["metadatas"], results["embeddings"]):
    print(f"ID: {id}, Contenido: {content[:100]} ", f"Metadata: {metadata}", f"Embedding: {embedding[:5]}...")

ID: 67429328-8888-4bfb-bc92-41efe8861e89, Contenido: Las montañas de los Andes se alzan majestuosamente a lo largo de la frontera occidental de Argentina  Metadata: {'source': 'doc2.txt'} Embedding: [0.06295843422412872, -0.008718427270650864, -0.0037864260375499725, -0.0077365185134112835, 0.018520746380090714]...
ID: 12318c8a-3668-48c7-97ba-16964cdaa097, Contenido:  folklóricas se mantienen vivas en este paisaje declarado Patrimonio de la Humanidad por la UNESCO.
  Metadata: {'source': 'doc2.txt'} Embedding: [0.06265807151794434, 0.07755861431360245, 0.006837757769972086, -0.0011955257505178452, 0.011039163917303085]...
ID: c1e519ce-1292-4da7-a901-1493d9619e19, Contenido: iares, con el famoso Perito Moreno, atrae a visitantes de todo el mundo.

La fauna argentina es dive  Metadata: {'source': 'doc2.txt'} Embedding: [0.04202388972043991, 0.01181267760694027, 0.03150956705212593, 0.006950356997549534, 0.02969401329755783]...
ID: 0671d6b1-752a-4c25-9fb6-2273dc2da798, Contenido: ósiles.


In [ ]:
while True:
    print(f"Total de documentos en la colección actualizado: {collection.count()}")
    opcion = input("Elige una opción o escribe ayuda: ")
    
    if opcion=="ayuda":
        print("\n--- Menú CRUD ---")
        print("1. Crear documento")
        print("2. Consultar por tema")
        print("3. Actualizar documento (por ID)")
        print("4. Eliminar documento (por ID)")
        print("5. Buscar documento (por ID)")
        print("6. Salir")

    if opcion == "1":
        nuevo_doc = input("Ingresa el texto a añadir: ")
        create_example(nuevo_doc)
        
    elif opcion == "2":
        consulta = input("Consulta: ")
        read_example(consulta)
        
    elif opcion == "3":
        doc_id = input("ID del documento a actualizar: ")
        nuevo_texto = input("Nuevo contenido: ")
        update_example(doc_id, nuevo_texto)
        
    elif opcion == "4":
        doc_id = input("ID del documento a eliminar: ")
        delete_example(doc_id)  

    elif opcion == "5":
        doc_id = input("ID del documento a buscar: ")
        get_documents_by_id(doc_id)
    
    elif opcion == "6":
        print("👋 Saliendo del programa.")
        break

Total de documentos en la colección actualizado: 36
❌ El documento está vacío. No se puede agregar.
Total de documentos en la colección actualizado: 36
❌ El documento está vacío. No se puede agregar.
Total de documentos en la colección actualizado: 36
